# Week 7: Model Evaluation 🔍
# Tutorial

In the previous week, we trained a model using the training data and improved the model by tuning hyperparameters. This week, we will use the test data to do a final evaluation of how well our model performs on unseen data.

## Learning objectives

We will discuss the following topics and how to use the evaluation metrics in `sklearn`:
1. Generalization
    * Case Study
3. Underfitting/overfitting
4. Evaluation metrics (covered in this week's pre-module exercise)
    * Accuracy
    * Confusion Matrix
    * Precision & Recall
    * F1-score

We will then evaluate our trained breast cancer model using our test set and interpret the results.

### Setup

First, **run the cell below** to load a trained model and the dataset (split into train and test). This model has been trained similarly to your work in week 6's tutorial.

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
import pickle

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

# Set random seed for reproducibility
np.random.seed(42)

df = pd.read_csv('bc_data.csv', index_col=0)

# Data cleaning
# Remove the 'Unnamed: 32' column
df = df.drop('Unnamed: 32', axis=1)

# Encode target feature to binary class and split target/predictor vars
y = df["diagnosis"].map({"B" : 0, "M" : 1})
x = df.drop("diagnosis", axis = 1)

# Drop all "worst" columns
cols = ['radius_worst', 
        'texture_worst', 
        'perimeter_worst', 
        'area_worst', 
        'smoothness_worst', 
        'compactness_worst', 
        'concavity_worst',
        'concave points_worst', 
        'symmetry_worst', 
        'fractal_dimension_worst']
x = x.drop(cols, axis=1)

# Drop perimeter and area (keep radius)
cols = ['perimeter_mean',
        'perimeter_se', 
        'area_mean', 
        'area_se']
x = x.drop(cols, axis=1)

# Data splitting
# train (70%), val (15%), test (15%)
# val set not needed in this module, but kept for consistency
# random_state seed is the same as week 6; resulting sets should be the same
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_ratio, random_state=40)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=val_ratio/(train_ratio + val_ratio), random_state=40)

# Step 1: Define the true underlying function (hidden from students)
def true_function(x):
    return 0.02 * (x-2) * x * (x-1) * (x+3) * (x-5) 

# Load a trained model
with open('bc_model.pkl', 'rb') as file:
    model = pickle.load(file)

### Generalization

When we talk about model performance, we want to know how well a model <span style="background-color: #AFEEEE">**generalizes**</span>. This measures a model's ability to perform well on **new, unseen data**. Remember that at the beginning of our pipeline, we created hold-out data that we never showed the model during training. We will use this test data in this evaluation stage.

To explore this concept, we will explore the concept of polynomial regression, similar to linear or logistic regression explored in Module 5. The idea of linear and polynomial regression is that we want to create a function that best *fits* the data we're looking at. With linear regression, we're finding the line of best fit. With polynomial regression, we can pick a degree for the polynomial and try to fit a polynomial of that degree to the data.

As a reminder, a polynomial is defined as $$y = a_0x^0 + a_1x^1 + a_2x^2 + \dots + a_n x^n$$ with $n$ denoting the degree of the polynomial. So, for example, $y = x^4 + x$ is a 4th degree polynomial, $y = x$ is a first degree polynomial, and $y = 6$ is a zeroth degree polynomial, or a **constant** function.

#### **Case Study**

A novel species of bacteria called *Clostridium twoohonesus* has been found to be responsible for causing a condition called **HMBosis**. This condition is characterized by a high enjoyment of the course content being learned in HMB201. The infected are more prone to taking HMB301 and HMB491 in the future.

The bacteria have novel biochemical interactions that are understood to be temperature-dependent, but not fully elucidated. Additionally, a subset of these interactions is known to heavily impact the growth rate of the bacteria.

You are a scientist tasked with modelling the growth rate of the bacteria as a function of temperature. You have incubated a culture of the bacteria at different temperatures, but didn't have access to [Pytri AI's](https://www.pytri.ca/) (an HMB491 partner!) automatic colony counter, and had to manually count the number of bacteria in each plate.

You have the following data to train your model:
| Temperature (C) | Colony Growth Rate |
|-----------------|--------------|
| - 2.888         |       0.57779262  |
| - 1.777         |        2.41393475  | 
| - 0.666         |        1.39521538   |
| 0.444           |       0.9104623    |
| 1.555           |       1.0518173    |
| 2.666           |        -1.62275662  |
| 3.777           |        -3.40006912  |
| 4.888           |        -0.63161147  |

And you also have the following data to test your model:
| Temperature (C) | Colony Growth Rate |
|-----------------|--------------|
| - 1.5           |          2.87371602  |
| 1               |          -1.10633497  |
| 3.5             |          -3.25558162  |

First, you will need to insert the data into lists.

In [ ]:
temperature_train = [-2.888, -1.777, -0.666, 0.444, 1.555, 2.666, 3.777, 4.888]
growth_rate_train = [0.57779262, 2.41393475, 1.39521538, 0.9104623, 1.0518173, -1.62275662, -3.40006912, -0.63161147]

temperature_test = [-1.5, 1, 3.5]
growth_rate_test = [2.87371602, -1.10633497, -3.25558162]

Now, plot the training data and the test data to get a sense of the data.

Make sure to use a grid and label the axes.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(temperature_train, growth_rate_train, label='Training data', color='blue')
plt.scatter(temperature_test, growth_rate_test, label='Test data', color='green')
plt.xlabel('Temperature (˚C)')
plt.ylabel('Colony Growth Rate')
plt.title('Colony Growth Rate vs Temperature')
plt.grid(True)
plt.legend()
plt.show()

Luckily for you, a kind colleague at your lab has provided you with the code to perform polynomial regression.

The function also reports the average loss of the model. Here, the loss is defined as the absolute difference between the predicted growth rate and the true growth rate at each temperature. This is a pretty good way to determine if our model is accurately predicting the growth rate at each temperature.

Your job is now to find the best-fit polynomial for the data. We want to **absolutely minimize the loss**, to get as close to 0 as possible!

In [ ]:
def polynomial_regression(x, y, degree):
    poly = PolynomialFeatures(degree=degree)
    x_poly = poly.fit_transform(np.array(x).reshape(-1, 1))
    
    model = LinearRegression()
    model.fit(x_poly, y)
    
    x_plot = np.linspace(-4, 6, 1000).reshape(-1, 1)
    x_plot_poly = poly.transform(x_plot)
    y_pred = model.predict(x_plot_poly)
    
    y_train_pred = model.predict(x_poly)
    
    losses = np.abs(y - y_train_pred)
    average_loss = np.mean(losses)
    print(f"Average Training Loss: {average_loss:.4f}")

    plt.figure(figsize=(10, 6))
    plt.scatter(x, y, label='Training data', color='blue')
    plt.plot(x_plot, y_pred, label=f'Polynomial fit (degree {degree})', color='red')
    
    for i in range(len(x)):
        plt.plot([x[i], x[i]], [y[i], y_train_pred[i]], 'k--', linewidth=0.8)
    
    plt.legend()
    plt.title(f'Colony Growth Rate vs Temperature (Polynomial Regression (degree {degree}))')
    plt.xlabel('Temperature (˚C)')
    plt.ylabel('Colony Growth Rate')
    plt.grid(True)
    plt.ylim(-5, 6)  # Restrict y-axis range
    plt.show()
    
    return average_loss

Try a few different degrees and pick which one **you** think is the best fit. Specifically, pick some low values and some high values.

In [ ]:
degree = 10
polynomial_regression(temperature_train, growth_rate_train, degree)

**Concept Check:** What happened when you used a low degree polynomial? What about a high degree polynomial? How did the curve change? How did the loss change?

**Answer:**
* Low degree: The curve did not fit the data well. At 1, specifically, we had a downward sloping line, which looked like it could fit the data well.
* High degree: The curve fits the training data very well, with a lot of ups and downs.

With a low-degree polynomial, we saw that the curve did not fit the data well, and had a higher loss. In this case, our model was '**underfitting**' the data. This is when the model is 'too simple' to fit the data well.

Now, we should also see how well our model performs on the test data. This will tell us how well our model generalizes to new data.

In [ ]:
def polynomial_regression_test(x_train, y_train, x_test, y_test, degree, true_f=False):
    poly = PolynomialFeatures(degree=degree)
    x_train_poly = poly.fit_transform(np.array(x_train).reshape(-1, 1))
    
    model = LinearRegression()
    model.fit(x_train_poly, y_train)
    
    x_test_poly = poly.transform(np.array(x_test).reshape(-1, 1))
    
    y_train_pred = model.predict(x_train_poly)
    y_test_pred = model.predict(x_test_poly)
    
    train_losses = np.abs(y_train - y_train_pred)
    test_losses = np.abs(y_test - y_test_pred)
    average_train_loss = np.mean(train_losses)
    average_test_loss = np.mean(test_losses)
    print(f"Average Training Loss: {average_train_loss:.4f}")
    print(f"Average Test Loss: {average_test_loss:.4f}")

    x_plot = np.linspace(-4, 6, 1000).reshape(-1, 1)
    x_plot_poly = poly.transform(x_plot)
    y_pred = model.predict(x_plot_poly)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(x_train, y_train, label='Training data', color='blue')
    plt.scatter(x_test, y_test, label='Test data', color='green')

    if true_f:
        plt.plot(x_plot, true_function(x_plot), label=f'True function', color='black')

    plt.plot(x_plot, y_pred, label=f'Polynomial fit (degree {degree})', color='red')
    
    for i in range(len(x_train)):
        plt.plot([x_train[i], x_train[i]], [y_train[i], y_train_pred[i]], 'b--', linewidth=0.8)
    
    for i in range(len(x_test)):
        plt.plot([x_test[i], x_test[i]], [y_test[i], y_test_pred[i]], 'g--', linewidth=0.8)
    
    plt.legend()
    plt.title(f'Colony Growth Rate vs Temperature (Polynomial Regression (degree {degree}))')
    plt.xlabel('Temperature (˚C)')
    plt.ylabel('Colony Growth Rate')
    plt.grid(True)
    plt.ylim(-5, 6)  # Restrict y-axis range
    plt.show()
    
    return average_train_loss, average_test_loss

In [ ]:
polynomial_regression_test(temperature_train, growth_rate_train, temperature_test, growth_rate_test, 5)

**Concept Check:** What happened when you used a low-degree polynomial? What about a high-degree polynomial? How did the curve change? How did the training and test loss change?

**Answer:** With a low-degree polynomial, the curve did not fit the data well, and the training and test losses were high. With a high degree polynomial, the curve fit the training data very well, but the test loss was high.

**Concept Check:** What degree polynomial is the best for modelling this data? Why?

**Answer:** I chose a degree 5 polynomial because the curve fit the training data well, and the test loss was lowest among all degrees.


You'll now have seen that using a very high-degree polynomial can perfectly fit the training data, but it may not generalize well to unseen test data. This is where the concept of **overfitting** comes in.

Overfitting is when a model is too complex and fits the training data too well, so that it captures noise in the training data. This can lead to poor generalization to unseen data.

We can see this in the graph above, where the curve fits the training data very well, but the test loss was high. This is because the model is too complex and has 'memorized' the training data, rather than learning the underlying pattern.

#### **Case Study Conclusion**

Finally, after investigating the model's performance, you've come to the conclusion that a degree 5 polynomial is the best fit for this data. You can now use this model to make predictions on new data.

Through investigations using Michaelis-Menten Kinetics, the growth rate-temperature relationship is found to be:

$$ y = 0.02 (x+3)x(x-2)(x-3)(x-5) $$

which is indeed a 5th-degree polynomial. Here is the function plotted with the training data and test data, and the predictions of the model:

In [ ]:
polynomial_regression_test(temperature_train, growth_rate_train, temperature_test, growth_rate_test, 5, True)

### Generalization Summary

**Noise**: irrelevant data in the dataset that is not part of the useful pattern that needs to be learned. Outliers, errors, or inconsistencies in the dataset are examples of noise. We say that a dataset is **noisy** if it includes many such datapoints. 

|  | Definition | How to detect |
| --- | --- | --- |
| **Underfitting** | a model is not able to capture the useful/important patterns in the dataset. | Poor performance on the training set | 
| **Overfitting** | a model performs poorly on unseen test data, even though it performed well during training. | Great performance on the training set and poor performance on the test set |

When the model performs well on the training set and the test set, we say that it is a **good fit** and **generalizes** well. This means the model has learned from the training set well enough to apply in the general case, to new data it has not seen before.

**Generalization**: a model's ability to adapt to and perform well on new, unseen data.
A model generalizes well if it performs well on unseen data (i.e., a test set)

### Generating model predictions

First, we need to get the trained model's predictions. Our evaluation metrics will need both the classification (0/1) labels and prediction probabilities. We can do this using the **`predict()`** function, which can be accessed and called on `sklearn` model objects. The model that you trained in week 6 is an object of the `SGDClassifier` class, so this function is available on your model.  

| Function | Input Parameters | Output | Syntax |
| --- | --- | --- | --- |
| `predict()` | array of input data points | predicted classifications (0's and 1's) corresponding to each input data point. For example, [0, 1, 1, 0, 0, 0, 1, 0, ... , 1] | `model.predict(input_testset)` |

Documentation for `SGDClassifier`: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

Scroll down to "methods" to see the functions that can be called on objects of this class.

---
**Q*1. Complete the code below to call `predict()` on the trained model, using your test set inputs.**

<span style="background-color: #FFD700">**Write your code below**</span> 

In [ ]:
# Your test set is saved in variables x_test and y_test.
# Your trained model is saved in the variable model.

# Prediction
preds = ...    #TODO: complete this line

print(preds)

---
### Using evaluation metrics

In the pre-module, we talked about 4 metrics we can calculate: accuracy, precision, recall, and the F1-score (along with the confusion matrix). 

To get these results using `sklearn`, we need to import some functions from `sklearn.metrics`.

**Run the code below to import the libraries for model metrics.**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

### Generating the Confusion Matrix

Let's generate the confusion matrix first, using the `confusion_matrix()` function provided in `scikit-learn`.

In this table, `y_true` = array of input data points and `y_pred` = array of predicted labels.

| Function | Input parameters | Output | Syntax |
| --- | --- | --- | --- |
| confusion_matrix() | y_true, y_pred | a confusion matrix represented as a 2D array | confusion_matrix(y_test, preds) |

---
**Q*2. Complete the code below to generate the confusion matrix.**

<span style="background-color: #FFD700">**Write your code below**</span> 

In [ ]:
conf = ...    #TODO: complete this line

---
### Confusion Matrix Display
Right now, if we print the confusion matrix, we get:

In [ ]:
print("Confusion matrix:\n",conf)

For better, clearer visualization with proper axis labels, we can use the `ConfusionMatrixDisplay` object from `sklearn`. 

First, we have to import a function from `sklearn.metrics`, like we did for all the other metrics. The `ConfusionMatrixDisplay()` function creates an object of the `ConfusionMatrixDisplay` class, and this object can display/plot the confusion matrix.

A little explanation on the input parameters to this function:
* `confusion_matrix`: a confusion matrix object (from `sklearn` `confusion_matrix`)
* `display_labels`: an array of axis labels. You can obtain them from your model with `model.classes_`

| Function | Input parameters | Output | Syntax |
| --- | --- | --- | --- |
| ConfusionMatrixDisplay() | confusion_matrix, display_labels | a display object for confusion matrix. | ConfusionMatrixDisplay(confusion_matrix, display_labels) |

Finally, calling **`plot()`** on your newly created confusion matrix display object will plot the actual display. 

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html

---
**Q*3. Complete the code below to create a confusion matrix display.**

<span style="background-color: #FFD700">**Write your code below**</span> 

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
labels = np.where(model.classes_, 'malignant', 'benign')

# TODO: complete this line
display = ...

display.plot()

---
**Q*4. Identify the number of FP, FN, TP, and TN from the confusion matrix of the breast cancer detection model. Malignant is positive.**

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer here: 

---

**Q*5. State the ratio of actual positives to actual negatives. Is this dataset balanced?**

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

**Q*6. Using what you learned in the pre-module, calculate the accuracy, precision, recall, and f1-score for the breast cancer model based on the confusion matrix.**

> Hint: Students can look at their completed pre-module work/the formulas presented in that notebook as a guide. 

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here: 

---

### Generating Accuracy, Precision, Recall, F1-Score

Here is a summary of the `sklearn` functions for the metrics we covered in the pre-module, as well as their input parameters, output, and syntax. Note that there are additional optional input parameters than listed here, but for this particular binary classification problem, we do not need them. We have only listed the required parameters for each function. 

In this table, `y_true` = array of input data points and `y_pred` = array of predicted labels.

| Function | Input parameters | Output | Syntax |
| --- | --- | --- | --- |
| accuracy_score() | y_true, y_pred | accuracy (decimal) | accuracy_score(y_test, preds) |
| precision_score() | y_true, y_pred | precision (decimal) | precision_score(y_test, preds) | 
| recall_score() | y_true, y_pred | recall (decimal) | recall_score(y_test, preds) | 
| f1_score() | y_true, y_pred | f1-score (decimal) | f1_score(y_test, preds) | 

---
**Q*7. Complete the code below to get each metric and save them to the given variables.**

<span style="background-color: #FFD700">**Write your code below**</span> 


In [ ]:
# Evaluation metrics
acc =         #TODO: complete this line
precision =   #TODO: complete this line
recall =      #TODO: complete this line
f1 =          #TODO: complete this line

---
Let's print all our results. **Run the code below**.

In [ ]:
eval = pd.Series({
    "Accuracy": acc,
    "Precision": precision,
    "Recall": recall,
    "F1-Score": f1
})

print("Model: LogisticRegression")
print("Evaluation metrics:")
print(eval)

Look at the generated evaluation results for the 4 metrics above. Compare them with your hand-calculated results in question 6 - they should be the same. You can use the `scikit-learn` generated values to check your work.

---
**Q*8. There is no universal threshold for what constitutes a "good model". However, your colleagues at the Breast Cancer Institute have unanimously decided that a model with an F1-score of 0.8 and above will suffice. They believe that using the F1-score to evaluate the model is the best approach, since it combines precision and recall, and the dataset is imbalanced. They are not considering using any other metrics at the moment.** 

**You disagree with your colleagues on this decision. How would you evaluate the performance differently, and why? What information might you need (from physicians, medical researchers, etc.) to make a recommendation?**

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

## **Graded Exercise: (5 marks)**

**GQ*1. (2 marks) Suppose that upon evaluating your model, you find that the accuracy is very low (around 60%). However, the accuracy during training was very good (around 95%). What do you call this phenomenon, and how would you try to correct it? List at least two methods discussed in this module.**

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

**GQ*2. (2 marks) Suppose that your model has an accuracy of 62% during training, using a logistic regression model and 200 training samples. After obtaining more samples for your dataset, you retrain the model with 400 training samples, but the accuracy only goes up to 66%. What do you call this phenomenon, and how would you try to correct it? List at least two methods discussed in the ML modules.**

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

**GQ*3. (1 mark) Say that one of your colleagues has trained a different model. Their evaluation results look like this:**

Colleague's evaluation results:
* Accuracy     0.868421
* Precision    0.900000
* Recall       0.692308
* F1-Score     0.782609

And your evaluation results look like this:
* Accuracy     0.837209
* Precision    0.695652
* Recall       1.000000
* F1-Score     0.820513

**Assume that the dataset is imbalanced (more benign cells). Your colleague says that your model performs better than theirs. Describe two reasons, seen in the two evaluation results, to support this claim.**

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer here:

---

## Conclusion

Let's go back to our motivation -- we set out to find out how well a machine learning model could be used to predict which cancer cells are benign or malignant, as a form of early diagnosis to improve patient care. What did we learn? Based on our evaluation, the logistic regression model we trained on this dataset shows potential to be useful as a tool to aid physicians in their diagnosis; an F1-score of 0.82 is generally considered good. However, more information from stakeholders might be needed to quantify exactly how "good" the model has to be for it to be feasible in a medical setting. Realizing that these decisions might require expert opinions and interdisciplinary collaboration, you call for a meeting with machine learning researchers, biologists, physicians, and your colleagues at the Canadian Cancer Society. You organize your results and learnings into a report, and head off to the meeting with valuable information that could leave a lasting impact on healthcare.

### Recap of ML Pipeline

<img src="image-8.png" alt="Drawing" style="width: 650px;"/>
<img src="image-9.png" alt="Drawing" style="width: 650px;"/>
<img src="image-11.png" alt="Drawing" style="width: 650px;"/>
<img src="image-12.png" alt="Drawing" style="width: 650px;"/>

Congratulations! You are now able to identify what happens in each stage of a machine learning pipeline and how each stage works. 